Duplikat --> Delete
Null --> Delete
Outlier --> Delete
Feature Selection --> K-Best

In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [2]:
# read csv
df = pd.read_csv('../UFC_train.csv')

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [3]:
# Delete duplicate
print(f"Shape before dropping duplicates : {df.shape}") # Before dropping duplicates
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicates: {df.shape}") # After dropping duplicates

Shape before dropping duplicates : (5410, 144)
Shape after dropping duplicates: (5410, 144)


In [4]:
# Delete null
print(f"Shape before dropping null : {df.shape}") # Before dropping null
df.dropna(inplace=True)
print(f"Shape after dropping null: {df.shape}") # After dropping null

Shape before dropping null : (5410, 144)
Shape after dropping null: (3480, 144)


In [6]:
# Delete outlier
print(f"Shape before dropping outlier : {df.shape}") # Before dropping outlier


Shape before dropping outlier : (3480, 144)


In [5]:
# Check outlier percentage in each column
def outlier_percentage(df: pd.DataFrame):
    for i in range(0, len(df.columns) - 1):
        column = df[df.columns[i]]
        if column.dtype == 'float64' or column.dtype == 'int64':
            q1 = column.quantile(0.25)
            q3 = column.quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - (1.5 * iqr)
            upper_bound = q3 + (1.5 * iqr)
            print(df.columns[i], ':', ((column <= lower_bound) | (column >= upper_bound)).sum() / len(df) * 100, '%')

outlier_percentage(df)

B_avg_KD : 2.327586206896552 %
B_avg_opp_KD : 8.67816091954023 %
B_avg_SIG_STR_pct : 2.6149425287356323 %
B_avg_opp_SIG_STR_pct : 2.3563218390804597 %
B_avg_TD_pct : 0.0 %
B_avg_opp_TD_pct : 2.442528735632184 %
B_avg_SUB_ATT : 5.373563218390805 %
B_avg_opp_SUB_ATT : 7.586206896551724 %
B_avg_REV : 16.580459770114942 %
B_avg_opp_REV : 15.344827586206897 %
B_avg_SIG_STR_att : 2.1551724137931036 %
B_avg_SIG_STR_landed : 2.21264367816092 %
B_avg_opp_SIG_STR_att : 2.6436781609195403 %
B_avg_opp_SIG_STR_landed : 2.7873563218390807 %
B_avg_TOTAL_STR_att : 1.5517241379310345 %
B_avg_TOTAL_STR_landed : 1.896551724137931 %
B_avg_opp_TOTAL_STR_att : 1.810344827586207 %
B_avg_opp_TOTAL_STR_landed : 2.2988505747126435 %
B_avg_TD_att : 4.741379310344827 %
B_avg_TD_landed : 5.172413793103448 %
B_avg_opp_TD_att : 4.626436781609195 %
B_avg_opp_TD_landed : 4.454022988505748 %
B_avg_HEAD_att : 3.074712643678161 %
B_avg_HEAD_landed : 3.1609195402298855 %
B_avg_opp_HEAD_att : 3.189655172413793 %
B_avg_opp_

In [7]:
# Drop outliers in x y z columns
def drop_outliers(df, column_name):
    # Calculate the IQR (Interquartile Range)
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter the DataFrame to exclude outliers
    df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    return df_filtered

numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
for column in numerical_columns:
    df = drop_outliers(df, column)

print("Shape after dropping outlier:", df.shape)

Shape after dropping outlier: (23, 144)


Bro..... Jangan pake kombinasi yang ini